adding the raw inference outputs to the cropped_mosaics

In [1]:
import rasterio
import numpy as np
from skimage.transform import resize

with open('data_manipulation/tile_names_inference.txt', 'r') as file:
    filenames = file.read().splitlines()

for filename in filenames:
    # Load the TIFF file
    with rasterio.open(f'../../notebook/cropped_mosaic_no_nan/{filename}_cropped_mosaic.tif') as src:
        # Read the existing bands
        bands = src.read()
        print("cropped_mosaic shape")
        print(bands.shape)

        # Load the NumPy array
        array = np.load(f'done_inference_tiles/inference_ensemble_{filename}.npy')
        print("inference_ensemble shape")
        print(array.shape)
        array = np.mean(array, axis=0, keepdims=True)
        data = array[0]
        print("averaged_inference shape")
        print(data.shape)
        data = np.moveaxis(data, 2, 0)

        data = data[:,7::15,7::15]
        print("downsampled data shape")
        print(data.shape)
        resized_data = resize(data, bands.shape)
        print("resized_data shape")
        print(resized_data.shape)

        # Check if the dimensions of the array match the TIFF file
        if bands.shape[1:] != resized_data.shape[1:]:
            raise ValueError("Array dimensions do not match the TIFF file")

        difference = np.where(bands[0] == -9999, 0 - resized_data[0], bands[0] - resized_data[0])
        difference = np.expand_dims(difference, axis=0)
        # Create a new array with the additional band
        new_bands = np.concatenate((bands, resized_data, difference), axis=0)

        profile = src.profile
        profile['count'] = 5

        '''
        band 1: icesat agbd
        band 2: icesat std
        band 3: predicted agbd
        band 4: ensemble std
        band 5: difference between icesat and predicted agbd
        
        '''

        # Create a new TIFF file with the additional band
        with rasterio.open(f'done_inference_tiles/inference_tifs/{filename}_inference.tif', 'w', **profile) as dst:
            dst.write(new_bands)
        print(f'======== {filename} done ========')


cropped_mosaic shape
(2, 3608, 3607)
inference_ensemble shape
(1, 10980, 10980, 2)
averaged_inference shape
(10980, 10980, 2)
downsampled data shape
(2, 732, 732)
resized_data shape
(2, 3608, 3607)
======== 52VEQ done ========
cropped_mosaic shape
(2, 3607, 3606)
inference_ensemble shape
(2, 10980, 10980, 2)
averaged_inference shape
(10980, 10980, 2)
downsampled data shape
(2, 732, 732)
resized_data shape
(2, 3607, 3606)
======== 52VFQ done ========
cropped_mosaic shape
(2, 3606, 3607)
inference_ensemble shape
(1, 10980, 10980, 2)
averaged_inference shape
(10980, 10980, 2)
downsampled data shape
(2, 732, 732)
resized_data shape
(2, 3606, 3607)
======== 52VDP done ========
cropped_mosaic shape
(2, 3606, 3606)
inference_ensemble shape
(2, 10980, 10980, 2)
averaged_inference shape
(10980, 10980, 2)
downsampled data shape
(2, 732, 732)
resized_data shape
(2, 3606, 3606)
======== 52VEP done ========
cropped_mosaic shape
(2, 3606, 3606)
inference_ensemble shape
(2, 10980, 10980, 2)
averaged_

FileNotFoundError: [Errno 2] No such file or directory: 'done_inference_tiles/inference_ensemble_52VEN.npy'